In [ ]:
library(reticulate)
library(plyr)
library(tidyr)
library(dplyr)
library(DALEX)
library(ggplot2)

In [ ]:
use_condaenv('analiza_dyfuzji', required = TRUE)
source_python('utils.py')
os <- import('os')

In [ ]:
model <- load_model('Models/model_hanna_2d.sav')

In [ ]:
data <- load_data('Data/characteristics/characteristics_h.csv', 1/6)

In [ ]:
class_vector <- unlist(CLASS_MAPPING, use.names = FALSE)

In [ ]:
x_train <- data[[1]]
x_test <- data[[2]]
y_train <- data[[3]]
y_test <- data[[4]]

### Create explainer

In [ ]:
explainer <- explain(model,
       data=x_train,
       y=y_train,
       label='Model',
       predict_function=custom_predict,
       residual_function=custom_residual_function)

In [ ]:
create_bd_plot <- function(explainer, x){
    bd <- predict_parts(explainer, x, type='break_down', N=500)
    bd$label <- mapvalues(bd$label, from=c(unique(bd$label)), to=unlist(CLASS_MAPPING, use.names = FALSE))
    bd
}

save_plot <- function(path, bd, height=20, width=8, max_features=17){
    plt <- plot(bd, max_features=max_features)
    pdf(path, height=height, width=width)
    print(plt)
    dev.off()
}

## Proces:
1. Dokonaj predykcji na zbiorze testowym;
2. Wybierz obserwacje o danym błędzie;
3. Wykonaj funkcję tworzącą bd-plot oraz dodaj wyniki do DF;
4. Zapisz wykres.

Funkcja podsumowująca:

Dla każdego z najczęstszych błędów i każdej cechy:

1. Rozkład cechy (na całym zbiorze);
2. Punktami zaznacz wartości cech, dla których występowały błędy, kolorem zaznacz wartości break-down;
3. Sprawdź średnie wartości break-down każdej z cech

### Predykcje na zbiorze testowym:

In [ ]:
y_pred <- model$predict(x_test)

In [ ]:
n_cols <- ncol(x_test)
df_predictions <- data.frame(y_real=y_test, y_pred=y_pred)
x_test_bd <- cbind(x_test, df_predictions)

In [ ]:
# Real vs Pred.
common_mistakes <- list(attm_sbm=c(0, 4),
                        attm_ctrw=c(0, 1),
                        fbm_sbm=c(2, 4))

### Wybranie obserwacji, bd-plot

In [ ]:
df <- data.frame()
for(el in common_mistakes){
    df_sub <- x_test_bd %>% filter(y_real==el[1],
                                   y_pred==el[2])
    df_sub <- df_sub[, 1:n_cols]
    for(row in 1:300){
        res <- create_bd_plot(explainer, df_sub[row, ])
        save_plot(os$path$join('Explanation',
                               'h',
                               'local',
                               'break_down',
                               paste0(class_vector[el[1]+1], '_', class_vector[el[2]+1]),
                                      paste0(row, '.pdf')),
                 res)
        to_spread <- res[(el[2]*(n_cols+2)+1):((el[2]+1)*(n_cols+2)), c(2, 3, 4)]
        to_add <- to_spread %>% pivot_wider(names_from='variable_name', values_from=c('contribution','variable_value'))
        to_add[, 'predicted_class'] <- el[2]
        to_add[, 'real_class'] <- el[1]
        df <- rbind(df, to_add)
        
    }
}

In [ ]:
write.csv(df, os$path$join('Data', 'break-down-values.csv'))